### Patterns in general
- "Each pattern describes a problem which occuurs over and over in our environment, and then describes the core of the solution to that problem, in such way that you can use this solution a million times over, without ever doing it the same way twice" [Alexander et al, "A Pattern Language"]
- Design (and thus patterns) is not independent from the implementation's technology.

### Classic SW DPs
- not data structures, nor algorithms
- not domain-specific architectures for entire subsystems

### DP write-up components
- NAME, context, problem
- KNOWN USES (KU): DP are discovered, not invented
- results, rationale, related DPs
- forces, solution, examples
- DP: description > prescription
- formal fixed schema not a must (but helpful)

## Categories of SQ DP
- __Creational__: concern the ways and means of object instantiation

- __Structural__: deal with the mutual composition of classes or objects

- __Behavioral__: analyze the ways in which classes or objects interact and distribute responsibilities among them

## Prolegomena
- Program to an interface, not to an implementation
- Favor object composition over class inheritance

    > - Strong Coupling -> inherit only when it's really convenient
    > - Not -> hold or wrap

## Hold & Wrap
- Hold: object O has subobject S as an attribute (maybe property)
    - self.S.method or O.S.method
    - Good: Simple, direct, immediate
    - Bad: Strong coupling, often on wrong axis
- Wrap: hold (via private name) plus deligation
    - O.method
    - Good: 
        - explicit (def method(self,...) <- self.S.method)
        - Automatic (delegation in `__getattr__`)
        - Gets coupling right (law of Demeter)\
- Inheritance cannot restrict

Python2: `raise E, V, T`
Python3: `raise E(V).with_traceback(T)`

In [4]:
class RestrictingWrapper(object):
    def __init__(self, wrap, block):
        self._wrap = wrap
        self._block = block
    def __getattr__(self, n):
        if n in self._block:
            raise AttributeError(n)
        return getattr(self._wrap, n)

# Creational Patterns
- not very common in Python because "factory" is essentially built-in

## Problem: We want just one instance to exist
1. \>90% times, use a module instead of a class
    - X: no subclassing, no special methods (`__add__`)...
2. Make just 1 instance (no enforcement)
3. Singleton
    - X: subclassing not really smooth
4. Monostate
    - X: Guido dislikes it
    
### Singleton
```Python
class Singleton(object):
    def __new__(cls, *a, **k):
        if not hasattr(cls, '_inst'):
            cls._inst = super(Singleton, cls).__new__(cls, *a, **k)
        return cls._inst
```
__CONS__: subclassing is a problem:
```Python
class Foo(Singleton): pass
class Bar(Foo): pass
f = Foo(); b = Bar(); # ...???...
```

### Monostate (Borg)
Everything will share the same state
```Python
class Borg(object):
    _shared_state = {}
    def __new__(cls, *a, **k):
        obj = super(Borg, cls).__new__(cls, *a, **k)
        obj.__dict__ = cls._shared_state
        return obj
```
subclassing:
```Python
class Foo(Borg): pass
class Bar(Foo): pass
class Baz(Foo): _shared_state = {}
```
_Data Overriding_ 

## Problem: Not commit to instantiating a specific concrete class
1. Dependency injection
    - no creation; injected from "outside"
    - X: Don't know how many widgets are needed / don't know how many dependencies to inject at the first place
2. Factory subcategory of DPs
    - may create whatever is approperate / reuse existing
    - factory functions (& other callables)
    - factory methods (overridable)
    - abstract factory classes
    
### Factories in Python
Each type/class is intrinsically a factory. Class is callable
- internally, may have `__new__` which can do anything
- externally, it's a callable, interchangeable with any other
- maybe injected directly (no need for boilerplate factory functions)

Modules can be kinda "abstract" factories without inheritance (e.g., `os` can be `posix` or `nt`)

#### Known Use: `type.__call__`: "two-phase construction: new-init"
```Python
def __call__(cls, *a, *k):
    nu = cls.__new__(cls, *a, **k)
    if isinstance(nu, cls):
        cls.__init__(nu, *a, **k)
    return nu
```

#### Factory-function example
```Python
def load(pkg, obj):
    m = __import__(pkg, {}, {}, [obj])
    return getattr(m, obj)
cls = load('p1.p2.p3', 'c4')
```

# Structural Patterns
Masquerading / Adaptation

- __Adapter__: Tweak an interface (both class and object variants)
- __Facade__: Simplify a subsystem's interface
- __Bridge__: let many implementations of an abstraction us many implementation of a functionality (without repetitive coding)
- __Decorator__: reuse + tweak without inheritance
- __Proxy__: decouple from access / location

## Adapter
- client code $\gamma$ requires a protocol $C$
- supplier code $\sigma$ provides different protocol $S$
    - with a superset of $C$'s functionality
- write adapter code $\alpha$ "sneaks in the middle"
    - to $\gamma$, $\alpha$ is a supplier (produces protocol $C$)
    - to $\sigma$, $\alpha$ is a client (consumes protocol $S$)
    - "inside", $\alpha$ implements $C$ (by means of appropriate calls to $S$ on $\sigma$)
    
### Toy example
- C requires foobar(foo, bar)
- S supplies method barfoo(bar, foo)

```Python
# Sigma
class Barfooer(object):
    def barfoo(self, bar, foo):
        pass
```

- Adapter at Object level
    - per-instance, with wrapping delegation
    
```Python
class FoobarWrapper(object):
    def __init__(self, wrappee):
        self.w = wrappee
    def foobar(self, foo, bar):
        return self.w.barfoo(bar, foo)
barfooer = Barfooer()
foobarer = FoobarWrapper(barfooer)
```

- Adapter at Class level
    - per-class, with subclasing and self-delegation
    - maybe faster but object level is cleaner
    
```Python
class Foobarer(Barfooer):
    def foobar(self, foo, bar):
        return self.barfoo(bar, foo)

foobarer = Foobarer()
```

### Adapter KU
- `socket._fileobject`: from sockets to file-like objects
    - socket: unbuffered, file: buffered
    - with much code for buffering
- `doctest.DocTestSuite`: adapts doctest tests to `unittest.TestSuite`
- `dbhash`: adapt dsbdb to dbm
- `StringIO`: adapt `str` or `unicode` to file-like
- `shelve`: adapt "limited dict" (str keys and values, basic methods) to complete mapping
    - via `pickle` for anything from/to string
    - `UserDict.DictMixin` to add other dict methods
    
### Adapter Observations
- Some real life adapters may require a lot more code
- mixin classes are a great way to help adapt to rich protocols
    - can implement advanced methods on top of fundamental ones
- Adapter occurs at all levels of complexity
- in Python, it's NOT just about classes and their instances (callables) 

## Facade
- Supplier code $\sigma$ provides rich, complex functionality in protocol $S$
- need simple subset $C$ of $S$
- facade code $\phi$ implements and supplies $C$
    - by means of appropriate calls to $S$ on $\sigma$
    
### Facade vs Adapter
- Adapter is about supplying a given protocol required by client-code, or gain polymorphism via homogeneity
- Facade is about simplifying a rich interface when just a subset is often needed
- Facade most often "fronts" for many objects, Adapter for just one

![Facade, before & after](Asset/Facade_before_after.png)

- With Facade: 
    - Life of Client is simpler: one simplified place
    - Life of Supplier is simpler: need only maintain one place when redoing a whole system 
    
### Facade KU
- `asynchat.fifo`: facades for list
- `dbhash` facades for `bsddb`, also given as example of Adapter
- old sets. `Set` used to facade for dict
- `Queue` facades for deque + lock
- `os.path`: basename, dirname facade for split + indexing ...

### Facade Observations
- Some real-life facades may have substantial code
    - simplifying the `_protocol_` is the key
    - interface simplifications are often accompanied by minor functional additions
- Facade occurs at all levels of complexity, but it;s most important for _complicated subsystems_ (and reasonably-simple views)
- inheritance is never useful for Facade, because inheritance can only "widen", never "restrict" => _wrapping_

## Adapting / Facading callables
- callables play a large role in Python programming
    - often need to adapt or facade callables
    - easily parse in arguments / return as results
- `functools.partial` to present some args (AKA "currying") + bound-methods special case 
- decorator syntax to adapt functions or methods by wrapping in HOFs (Higher Order Functions) (adding something)
- closures (nested functions) for simple HOF needs
    - classes with `__call__` for complex ones

## Bridge
__Problem__: Have $N1$ realizations $\rho$ of abstraction $A$, each using any one of $N2$ implementations $\iota$ of functionality $F$, _without_ coding $N1*N2$ cases
- have abstract superclass A of all $\rho$ hold a reference $R$ to the interface $F$ of all $\iota$,
- ensure each $\rho$ uses any functionality of $F$ (thus, from a $\iota$) _only_ by delegating to $R$
- i.e., each $\rho$ has to go through $R$ to reach $F$

### Toy Bridge Example
```Python
class AbstractParser(object):
    def __init__(self, scanner):
        self.scanner = scanner
    def __getattr__(self, name):
        return getattr(self.scanner, name)
class ExpressionParser(AbstractParser):
    def expr(self):
        token = self.next_token()
        self.push_back(token)
```

### Bridge KU: SocketServer
- `BaseServer` is the "abstraction" $A$
- `BaseRequestHandler` is $F$ (the abstract superclass for functionality implementation)
    - also uses mixins for threading, forking...
- note: $A$ holds the very class $F$ and instantiates it per-request (shades of a Factory DP) -- typical in Bridge DP in python (also e.g., email: Parser Parser -> message)
- Bridge is typical of somewhat "rich" complicated situations.



## Decorator
__Problem__: Client code $\gamma$ requires a protocol $C$; supplier code $\sigma$ does provide protocol $C$, but we want to insert some _semantic tweak_ (often dynamically plug-in / plug-out-able)
- Decorator code $\delta$ "sneaks in the middle":
    - $\gamma$ uses $\delta$ just as it would use $\sigma$
    - $\delta$ wraps $\sigma$, and it may intercept, modify, add (a little), delegate, ...
- Like an Adapter except _incoming_ and _outgoing_ interfaces are identical

### Toy Example Decorator
```Python
class WriteFullLinesOnlyFile(object):
    def __init__(self, *a, **k):
        self._f = open(*a, **k)
        self._b = ''
    def write(self, data):
        lns = (self._b + data).splitlines(True)
        if lns[-1][-1] == '\n':
            self._b = ''
        else:
            self._b = lns.pop(-1)
        self._f.writelines(lns)
    def __getattr__(self, name):
        return getattr(self._f, name)
```

### Decorator KU
- `gzip.GzipFile` decorates a file with compress / decompress functionality
- `threading.RLock` decorates `thread.Lock` with reentrancy & ownership functionality
- `codecs` classes doctorates a file with generic encoding and decoding functionality


## Proxy

__Problem__: client code $\gamma$ needs to access an object $\tau$. However, something interferes with that:
    - $\tau$ lives remotely, or in persisted form
    - access restrictions may apply (security)
    - lifetime or performance issues
    
- Proxy object $\pi$ "sneaks in the middle":
    - $\pi$ wraps $\tau$, may create / delete it at need
    - may intercept, call, delegate, ...
    - $\gamma$ uses $\pi$ as it would use $\tau$
    
### Toy Example Proxy
```Python
class RestrictingProxt(object):
    def __init__(self, block, f, *a, **k):
        self._mameit = f, a, k
        self._block = block
    def __getattr__(self, name):
        if name in self._block:
            raise AttributeError(name)
        if not hasattr(self, '_wrapped'):
            f, a, k = self._makeit
            self._wrapped = f(*a, **k)
        return getattr(self._wrapped, name)
```

### Proxy KU
- the values in a `shelve.Shelf` proxy for persisted objects (get instantiated at need)
- `weakref.proxy` proxies for any existing object but doesn't "keep it alive"
- `idlelib.RemoteDebugger` uses proxies (for frames, code objects, dicts and a debugger object) across RPC to let a Python process be debugged from a separate GUI process

# Behavioral Patterns

- Template Method: self-delegation
    - The essence of OOP
- State and Strategy as "factored out" extensions to Template Method

## Template Method
Template: Self-delegation (directly descriptive)
    - Template Method tends to imply more "organization"
    
### Classic TM
- abstract base class offers "organizing method" which calls "hook methods"
- in ABC, hook methods stay abstract
- concrete subclasses implement the hooks
- client code calls organizing method on some reference to ABC (injector, ...) which refers to a concrete subclass (SC) 

### TM Skeleton
```Python
class AbstractBase(object):
    def orgMethod(self):
        self.doThis()
        self.doThat()
class Concrete(AbstractBase):
    def doThis(self): pass
    def doThat(self): pass
```

### TM example: paginate text
To paginate text, one must:
- remember max number of lines/page
- _output each line_, while tracking the position on the page
- just before the first line of each page, _emit a page header_
- just after the last line of each page, _emit a page footer_

```Python
class AbstractPager(object):
    def __init__(self, mx=60):
        self.mx = mx
        self.cur = self.pg = 0
    def writeLine(self, line):
        if self.cur == 0:
            self.doHead(self.pg)
        self.doWrite(line)
        self.cur += 1
        if self.cur >= self.mx:
            self.doFoot(self.pg)
            self.cur = 0
            self.pg += 1
```
```Python
class PagerStdout(AbstractPager):
    """Concrete pager (stdout)"""
    def doWrite(self, line):
        print(line)
    def doHead(self, pg):
        print('Page %d: \n\n' % pg+1)
    def doFoot(self, pg):
        print('\f') # form-feed character
```
```Python
class PagerCursers(AbstractPager):
    """
    Concrete pager (curses): Simplified 'more'
    w: window
    """
    def __init__(self, w, mx=24):
        AbstractPager.__init__(self, mx)
        self.w = w
    def doWrite(self, line):
        self.w.addstr(self.cur, 0, line)
    def doHead(self, pg):
        self.w.move(0, 0)
        self.w.clrtobot()
    def doFoot(self, pg):
        self.w.getch() # wait for keypress
```

### Classic TM Rationale
- the "organizing method" provides "structural logic" (sequencing &c)
- the "hook methods" perform "actual 'elementary' actions"
- it's an often-appropriate factorization of commonality and variation
    - focuses on objects' (classes') responsibilities and collaborations
        - base class calls hooks
        - subclass supplies them
    - applies the "Hollywood Principle": "don't call us, we will call you"
    
Framework approach of OOP:
Not code I write calls functions from library but code provided by the framework calls concrete methods in my own objects



* Dependency Injection Myth: Reference Passing
  http://misko.hevery.com/2008/10/21/dependency-injection-myth-reference-passing/
* The Factory Method Pattern
  https://python-patterns.guide/gang-of-four/factory-method/